In [2]:
import os

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import pandas as pd
from PIL import Image

from datetime import datetime
import requests
from io import BytesIO
from tqdm import tqdm
from heic2png import HEIC2PNG
from PIL import Image

In [3]:
def get_image_data (
    image_url, text_cap_en=None, text_cap_native=None, text_culture_loc=None, 
    text_image_loc=None, text_submitter_native_lang=None
):
    created_at = datetime.now().strftime("%d/%m/%Y 00:00:00")
    return {
        'annotation_id': None, 'annotator': None, 'email': 'seacrowd.research@gmail.com', 
        'created_at': created_at, 'timestamp': created_at, 'id': f'bulk_{idx}', 'image': image_url, 
        'lead_time': None, 'text_cap_en': text_cap_en, 'text_cap_native': text_cap_native, 
        'text_culture_loc': text_culture_loc, 'text_image_loc': text_image_loc, 
        'text_submitter_native_lang': text_submitter_native_lang, 'updated_at': None
    }

In [7]:
BASE_PATH = "./sea-vl-image-collection/data"
GH_BASE_PATH = "https://raw.githubusercontent.com/SEACrowd/sea-vl-image-collection/refs/heads/main/data"
df = pd.read_csv('bulk_migrate_20250206.csv')
migration_data = []

for idx, row in tqdm(df.loc[df['migrate_date'].isna(), :].iterrows()):
    image_path = row['Image Upload']
    text_cap_en = row['In English, what is this image about?']
    text_cap_native = row['In your native language, what is this image about?']
    text_culture_loc = row['(Choose at least 1) This image portrays culturally-relevant information in...']
    text_image_loc = row['Where was this image taken? (City, Country)']
    text_submitter_native_lang = row["What's your native language?"]

    migration_data.append(
        get_image_data(image_path, text_cap_en, text_cap_native, text_culture_loc, text_image_loc, text_submitter_native_lang)
    )

###
# Image Dedup
### 

# TODO: DeDup image data
#  - Check all images from `sea-vl-image-collection/data`, exclude the one in `migration_data`
#  - Filter `migration_data` whenever there is duplication
#  - 

migrate_df = pd.DataFrame(migration_data)
migrate_df.to_csv(f'bulk_ready_to_upload_{datetime.now().strftime("%Y%m%d")}.csv', index=False)

545it [00:00, 9403.22it/s]


In [8]:
migrate_df.shape

(545, 14)